# ECG Arrhythmia detection

This project aims to demonstrate the Anomaly Detection module implemented in Darts
### Dataset
The MIT-BIH Supraventricular Arrhythmia Database (SVDB) contains 2 channels, and 78 half-hour ECG recordings obtained from 47 objects between 1975-1979.

### Task
Develop an anomaly detection model to identity arrhythmia in the ECG signal.

## Downloading ECG dataset

In [1]:
# Download data from data source to a local holder
import io
import os
import zipfile
import requests

# URL of the zip file
zip_url = "https://my.hidrive.com/api/sharelink/download?id=lmCmAjUP"

# Folder path to save the downloaded zip file
folder_path = "data"

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

file_path = os.path.join(folder_path, "svdb.zip")
if not os.path.exists(file_path):
    # Send a GET request to download the zip file
    response = requests.get(zip_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Save the zip file to the local drive
        with open(file_path, "wb") as file:
            file.write(response.content)
        print("Zip file downloaded successfully.")
else:
    print("Zip file already downloaded.")    

# Extract the zip file
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(folder_path)
print("Zip file extracted successfully.")

Zip file already downloaded.
Zip file extracted successfully.


## Install required libaries for anomaly detection

In [2]:
# Install the library darts
!pip install "darts @ "
!git clone https://github.com/unit8co/darts.git
!pip install darts/. -q

Cloning into 'darts'...
remote: Enumerating objects: 23033, done.
remote: Counting objects: 100% (4689/4689), done.
remote: Compressing objects: 100% (1314/1314), done.
^Cceiving objects:  20% (4607/23033), 72.10 MiB | 15.88 MiB/s
fetch-pack: unexpected disconnect while reading sideband packet
ERROR: Invalid requirement: 'darts/.'
Hint: It looks like a path. File 'darts/.' does not exist.


## Task #1
### Load data of a patient into a darts timeseries object

In [3]:
patient_number = '842'

In [4]:
from darts import TimeSeries

# Load data into darts TimeSeries object
timeseries = TimeSeries.from_csv(f"./{folder_path}/multivariate/SVDB/{patient_number}.test.csv", time_col='timestamp')
ts_ecg = timeseries[['ECG1','ECG2']]
ts_anomaly = timeseries['is_anomaly']


KeyboardInterrupt



### Visualize signal and anomaly

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1, figsize=(10,5))
ts_ecg['ECG1'].plot(ax=ax,label='ECG1', lw=0.5)
(ts_anomaly-2).plot(ax=ax,label='is_anomaly',color='r', lw=0.5)

## Task #2
### Identify a region of ~15000 datapoints with anomalies (Otherwise training takes for a while) and visualize it

In [ ]:
# Identify a subset for demonstration
start, end = 15000, 30000
ts_ecg_subset= ts_ecg[start:end]
ts_anomaly_subset = ts_anomaly[start:end]

In [ ]:
# Visualize the subset
fig,ax = plt.subplots(figsize=(10, 5))
ts_ecg_subset['ECG1'].plot(label='ECG1', lw=1.)
((ts_anomaly_subset/2)-1.5).plot(label='is_anomaly', color='r')

## Task #3
### Create training and test set size (e.g., 10k/5k, 12k/2k datapoints)

In [ ]:
# Create train and test dataset for demonstration
train_end, test_end = 10000, 15000 
ts_ecg_train = ts_ecg_subset[:train_end]
ts_ecg_test = ts_ecg_subset[train_end:test_end]
ts_anomaly_test = ts_anomaly_subset[train_end:test_end]

In [ ]:
# Visualize the train / test set as well as the test set anomalies
fig,ax = plt.subplots(figsize=(10, 5))
ts_ecg_train['ECG1'].plot(label='train', lw=1.)
ts_ecg_test['ECG1'].plot(label='test', lw=1.)
((ts_anomaly_test/2)-1.5).plot(label='is_anomaly', color='r')

## Task #4
### Assess data properties such as periodicity and identify most common period.

In [ ]:
from darts.utils.statistics import plot_acf, check_seasonality

# Visualise signal auto correlation to identify most common periodicity
plot_acf(ts=ts_ecg_subset['ECG1'], max_lag=220, m=92)

In [ ]:
# Identified most common period
period = 92

## Task #5
### Develop an anomaly detection model step by step by (see figure below):
1. Creating a forecasting model based on the train timeseries ECG data
2. Create historical forecasting for the test timeseries ECG data
3. Create anomaly scores using 2 different scores based on the forecasted and actual ECG signal

<img src="images/ad_inside_anomaly_model.png" alt="Image" width="60%" height="60%">

Links:
- Forecasting models: https://unit8co.github.io/darts/generated_api/darts.models.forecasting.html
- Scorers: https://unit8co.github.io/darts/generated_api/darts.ad.scorers.html?highlight=scorer#

#### Create a Forecasting model

In [ ]:
from darts.models import LinearRegressionModel

# Instatiate of a forecasting model - e.g. RegressionModel with a defined lag
forecasting_model = LinearRegressionModel(lags=period)

# Train the forecasting model on the training dataset
forecasting_model.fit(ts_ecg_train)

In [ ]:
# Visualization of predicted and actual signal
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(10,10))

# historical predictions
ts_ecg_test_predicted = forecasting_model.historical_forecasts(ts_ecg_test, retrain=False)
# residuals of these predictions = (y_true - y_pred)
ts_ecg_residuals = forecasting_model.residuals(ts_ecg_test, historical_forecasts=ts_ecg_test_predicted)

ts_ecg_test_predicted['ECG1'].plot(ax=ax1, label='ECG1_predicted')
ts_ecg_test['ECG1'].plot(ax=ax1, color='r', label='ECG1_test')
ts_ecg_residuals['ECG1'].plot(ax=ax2, color='b', label='Difference')

#### Use a NormScorer for scoring

In [ ]:
from darts.ad.scorers import NormScorer
scorer = NormScorer(ord=1, component_wise=False)

In [ ]:
# Calculate anomaly scores by taking the pointwise norm 1 (L1) and visualize the score
scores =  scorer.score_from_prediction(series=ts_ecg_test, pred_series=ts_ecg_test_predicted)
scores.plot(label='Anomaly Score')

In [ ]:
# Evaluate the calculated anomaly score using utility methods in darts
from darts.ad.utils import eval_metric_from_scores
eval_metric_from_scores(
    pred_scores=scores, 
    anomalies=ts_anomaly_test, 
    window=1, 
    metric='AUC_ROC'
)

#### Use fittable KMeansScorer for scoring
Norm scorer point wise calculates the norm between the predicted and actual timeseries. Since predicting th peak of the ECG signals are challenging for the model - the biggest differences can between the actual and predicted values are mostly found at the peak locations.

To overcome this issue stemming from the point wise comparison, a windowing approach inherent in the KMeanScorer will be used.
The KMeanScorer is a trainable scorer - meaning -  on the anomaly free dataset it fits 'k' centroids on the vectors/chunks of the ECG signal created by using a sliding window with width of 'w'. The scoring is done by determining the closest centroid distance to any future ECG window with the same size 'w'. Since the training was done on anomaly free windows, those windows which are containing anomalies will have larger distances even from the closest centroid.

The KMeanScorer as the figure illustrates below can directly used on the timeseries itself, however we will use the previously developed forecasting model to create historical prediction for the train dataset and do the training of the KMeanScorer on the absoule difference of the actual training and forecasted training dataset.

#### Training & Scoring
<img src="images/kmeansscorer.png" alt="Image" width="70%" height="70%">

<img src="images/ad_windowing.png" alt="Image" width="70%" height="70%">

In [ ]:
from darts.ad.scorers import KMeansScorer
scorer = KMeansScorer(k=50, window=2*period, component_wise=False, random_state=42)

In [ ]:
# Create historical forecasting on the trianing dataset for the scorer to be able to train
ts_ecg_train_predicted = forecasting_model.historical_forecasts(ts_ecg_train, retrain=False)

In [ ]:
scorer.fit_from_prediction(series=ts_ecg_train, pred_series=ts_ecg_train_predicted) 

#### Remark

The function `diff_fn` passed as a parameter to the scorer, will transform pred_series and actual_series into one series. `diff_fn` can be any of Darts ["per time step" metrics](https://unit8co.github.io/darts/generated_api/darts.metrics.html). By default, it compute the absolute difference (`darts.metrics.ae`). If `pred_series` and `actual_series` are lists of series, `diff_fn` will be applied to all pairwise elements of the sequences.


In [ ]:
# Use the trained scorer on the forecasted and actual test dataset
scores = scorer.score_from_prediction(series=ts_ecg_test, pred_series=ts_ecg_test_predicted)
scores.plot(label='Anomaly Score')

In [ ]:
# Calculate the performance on the scorer using built-in utils methods in darts
eval_metric_from_scores(
    pred_scores=scores,
    anomalies=ts_anomaly_test,
    window=2*period,
    metric='AUC_ROC'
)

## Task #6
### Develop the anomaly detection models by using the Forecasting Anomaly Model via dedicated Darts API interface
This exercise aims to illustrate the power of the darts anomaly detection module by hiding all of the previously made steps under the hood into one dedicated AnomalyModel and corresponding APIs

In [ ]:
#### Train an

In [ ]:
from darts.ad.anomaly_model.forecasting_am import ForecastingAnomalyModel
from darts.ad.scorers import NormScorer, KMeansScorer

# Instantiate the anomaly model with: one forecasting model, and one or more scorers (and corresponding parameters)
anomaly_model = ForecastingAnomalyModel(
    model=forecasting_model,
    scorer=[
         NormScorer(ord=1),
         KMeansScorer(k=50, window=2*period, component_wise=False, random_state=42)
    ],
)

# Fit anomaly model
anomaly_model.fit(ts_ecg_train, allow_model_training=False)

#### Create anomaly scores and prediction in one step


In [ ]:
# Score with the anomaly model (forecasting + scoring)
anomaly_scores, predictions = anomaly_model.score(
    series=ts_ecg_test, return_model_prediction=True
)

### Visualize the forecasted signal as well as the anomalies

In [ ]:
# Forecasting
ts_ecg_test['ECG1'].plot(label='test', lw=1.)
predictions['ECG1'].plot(label='prediction', lw=1.)

In [ ]:
# Anomalies
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5))
anomaly_scores[0].plot(ax=ax1, label="NormScorer") # indeces corresponding to the scorers
anomaly_scores[1].plot(ax=ax2, label="KMeansScorer") # indeces corresponding to the scorers

### Leverage the inbuilt darts visualization tool to evaluate and show anomalies

In [ ]:
# Visualize and evalute detection of anomalies
anomaly_model.show_anomalies(
    series=ts_ecg_test,
    anomalies=ts_anomaly_test,
    metric="AUC_ROC",
)
#plt.show()

## Task #7
Use a detector to binarize the anomaly scores

Link:
- https://unit8co.github.io/darts/generated_api/darts.ad.detectors.html?highlight=detector

In [ ]:
from darts.ad.detectors import QuantileDetector, ThresholdDetector

# Instantiate a detector
detector =  QuantileDetector(low_quantile=0, high_quantile=0.70)

fig, ax = plt.subplots()
(detector.fit_detect(anomaly_scores[0])-0).plot(lw=1., label='NormScorer - detected_anomaly')
(detector.fit_detect(anomaly_scores[1])-2).plot(lw=1., label='KMeanScorer - detected_anomaly')
(ts_anomaly_test-4).plot(color='r', lw=1, label='is_anomaly')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

In [ ]:
# Instantiate a detector
detector =  ThresholdDetector(low_threshold=0, high_threshold=0.7)

fig, ax = plt.subplots()
(detector.detect(anomaly_scores[0])-0).plot(lw=1, label='NormScorer - detected_anomaly')
(detector.detect(anomaly_scores[1])-2).plot(lw=1, label='KMeanScorer - detected_anomaly')
(ts_anomaly_test-4).plot(color='r', lw=1, label='is_anomaly')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')